In [1]:
# Pipeline 9 — Celda 1: entorno e imports
!pip -q install timm==1.0.9 --no-cache-dir

import os, sys, json, time, math, random, shutil
from dataclasses import dataclass
from pathlib import Path
from typing import Optional, Tuple, List

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import timm
from PIL import Image
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix

def set_seed(seed: int = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)
if device.type == "cuda":
    print("GPU:", torch.cuda.get_device_name(0))
else:
    print("⚠️ Activa GPU: Runtime > Change runtime type > GPU")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 60.5 MB/s eta 0:00:00
Device: cuda
GPU: Tesla T4


In [5]:
# --- Paths base y de salida ---
BASE_DIR   = Path("/content/drive/MyDrive/CognitivaAI")
DATA_DIR   = BASE_DIR / "oas1_data"
OUT_DIR    = BASE_DIR / "ft_effb3_stable_colab"   # carpeta de resultados estable
GRAPHS_DIR = OUT_DIR / "graphs_from_metrics"

# Crear directorios si no existen
OUT_DIR.mkdir(parents=True, exist_ok=True)
GRAPHS_DIR.mkdir(parents=True, exist_ok=True)

print("Device :", device)
print("GPU    :", torch.cuda.get_device_name(0) if device.type == "cuda" else "CPU")
print("BASE   :", BASE_DIR)
print("DATA   :", DATA_DIR, "| exists:", DATA_DIR.exists())
print("OUT    :", OUT_DIR)
print("GRAPHS :", GRAPHS_DIR)


Device : cuda
GPU    : Tesla T4
BASE   : /content/drive/MyDrive/CognitivaAI
DATA   : /content/drive/MyDrive/CognitivaAI/oas1_data | exists: True
OUT    : /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab
GRAPHS : /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/graphs_from_metrics


In [2]:
# Pipeline 9 — Celda 2: montaje Drive y rutas
from google.colab import drive
try:
    drive.mount('/content/drive', force_remount=False)
except Exception as e:
    print("⚠️ Drive ya estaba montado o la carpeta no está vacía.")

# Carpetas del proyecto
BASE      = Path("/content/drive/MyDrive/CognitivaAI")
DATA      = BASE / "oas1_data"
OUT       = BASE / "ft_effb3_stable_colab"   # <- carpeta específica del Pipeline 9
GRAPHS    = OUT  / "graphs_from_metrics"

# CSV mapeados (ajusta si tuvieras otros nombres)
VAL_MAP   = DATA / "oas1_val_colab_mapped.csv"
TEST_MAP  = DATA / "oas1_test_colab_mapped.csv"

# Crear carpetas de salida
OUT.mkdir(parents=True, exist_ok=True)
GRAPHS.mkdir(parents=True, exist_ok=True)

print("BASE   :", BASE)
print("DATA   :", DATA, "| exists:", DATA.exists())
print("OUT    :", OUT)
print("GRAPHS :", GRAPHS)
print("VAL_MAP:", VAL_MAP, "| exists:", VAL_MAP.exists())
print("TEST_MAP:", TEST_MAP, "| exists:", TEST_MAP.exists())


Mounted at /content/drive
BASE   : /content/drive/MyDrive/CognitivaAI
DATA   : /content/drive/MyDrive/CognitivaAI/oas1_data | exists: True
OUT    : /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab
GRAPHS : /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/graphs_from_metrics
VAL_MAP: /content/drive/MyDrive/CognitivaAI/oas1_data/oas1_val_colab_mapped.csv | exists: True
TEST_MAP: /content/drive/MyDrive/CognitivaAI/oas1_data/oas1_test_colab_mapped.csv | exists: True


In [3]:
# Pipeline 9 — Celda 3: config + carga mapeos + split por paciente
from dataclasses import dataclass
from typing import Tuple, Dict
import pandas as pd
import numpy as np
from pathlib import Path
import json, re, os, random

@dataclass
class CFG:
    img_size: int = 300
    batch_size: int = 64
    num_workers: int = 2     # T4 suele ir fino con 2; evita warnings de exceso
    seeds: Tuple[int,...] = (42,)
    holdout_patients: int = 10  # nº de pacientes para holdout desde VAL
    # Nota: TEST ya viene predefinido por oas1_test_colab_mapped.csv

cfg = CFG()
print(cfg)

# --- utilidades ---
def _standardize_cols(df: pd.DataFrame) -> pd.DataFrame:
    """Asegura columnas: patient_id, y_true, png_path (acepta variantes comunes)"""
    df = df.copy()
    # normaliza posibles nombres
    colmap = {c.lower(): c for c in df.columns}
    # mapeos candidatos
    pid_col  = next((c for c in df.columns if c.lower() in ["patient_id","patient","pid","subject"]), None)
    y_col    = next((c for c in df.columns if c.lower() in ["y_true","target","label","y"]), None)
    path_col = next((c for c in df.columns if c.lower() in ["png_path","path","filepath","file","img","image"]), None)

    assert pid_col is not None,  f"No encuentro columna de paciente en {df.columns.tolist()}"
    assert y_col   is not None,  f"No encuentro columna de etiqueta en {df.columns.tolist()}"
    assert path_col is not None, f"No encuentro columna de rutas en {df.columns.tolist()}"

    out = df[[pid_col, y_col, path_col]].rename(
        columns={pid_col:"patient_id", y_col:"y_true", path_col:"png_path"}
    ).reset_index(drop=True)

    # asegurar enteros 0/1
    out["y_true"] = out["y_true"].astype(float).round().astype(int)

    # quitar filas sin ruta válida
    out = out[out["png_path"].astype(str).str.len()>0].reset_index(drop=True)
    return out

def _load_mapped_csv(path: Path) -> pd.DataFrame:
    assert path.exists(), f"CSV no existe: {path}"
    df = pd.read_csv(path)
    df = _standardize_cols(df)
    # filtra filas cuyo archivo existe (Drive puede tener rutas huérfanas)
    exists_mask = df["png_path"].apply(lambda p: Path(p).exists())
    miss = (~exists_mask).sum()
    if miss:
        print(f"⚠️ Aviso: {miss} rutas no existen en disco. Se ignoran.")
    df = df.loc[exists_mask].reset_index(drop=True)
    return df

# --- cargar VAL y TEST mapeados ---
val_df  = _load_mapped_csv(VAL_MAP)
test_df = _load_mapped_csv(TEST_MAP)

def _patients(df: pd.DataFrame) -> int:
    return df["patient_id"].nunique()

print(f"VAL mapeado : shape={val_df.shape}, pacientes={_patients(val_df)}, y_mean={val_df['y_true'].mean():.3f}")
print(f"TEST mapeado: shape={test_df.shape}, pacientes={_patients(test_df)}, y_mean={test_df['y_true'].mean():.3f}")

# --- split por paciente: train/holdout desde VAL ---
# tomamos pacientes únicos de VAL y reservamos 'cfg.holdout_patients' para holdout
rng = np.random.RandomState(42)
val_pats = sorted(val_df["patient_id"].unique().tolist())
assert len(val_pats) >= cfg.holdout_patients + 1, "Muy pocos pacientes en VAL para reservar holdout."

rng.shuffle(val_pats)
holdout_pats = set(val_pats[:cfg.holdout_patients])
train_pats   = set(val_pats[cfg.holdout_patients:])

train_df   = val_df[val_df["patient_id"].isin(train_pats)].reset_index(drop=True)
holdout_df = val_df[val_df["patient_id"].isin(holdout_pats)].reset_index(drop=True)

# sanity checks
assert set(train_df["patient_id"]).isdisjoint(set(holdout_df["patient_id"])), "Mezcla de pacientes entre train/holdout"
assert set(test_df["patient_id"]).isdisjoint(set(train_df["patient_id"]).union(set(holdout_df["patient_id"]))), "Test comparte pacientes con train/holdout"

print(f"train_df  : {train_df.shape} | pacientes={_patients(train_df)} | y_mean={train_df['y_true'].mean():.3f}")
print(f"holdout_df: {holdout_df.shape} | pacientes={_patients(holdout_df)} | y_mean={holdout_df['y_true'].mean():.3f}")
print(f"test_df   : {test_df.shape} | pacientes={_patients(test_df)} | y_mean={test_df['y_true'].mean():.3f}")

# muestra rápida
print("\nEjemplo train_df:")
display(train_df.head(3))
print("\nEjemplo holdout_df:")
display(holdout_df.head(3))
print("\nEjemplo test_df:")
display(test_df.head(3))


CFG(img_size=300, batch_size=64, num_workers=2, seeds=(42,), holdout_patients=10)
VAL mapeado : shape=(940, 3), pacientes=47, y_mean=0.426
TEST mapeado: shape=(940, 3), pacientes=47, y_mean=0.426
train_df  : (740, 3) | pacientes=37 | y_mean=0.405
holdout_df: (200, 3) | pacientes=10 | y_mean=0.500
test_df   : (940, 3) | pacientes=47 | y_mean=0.426

Ejemplo train_df:


,patient_id,y_true,png_path
0,OAS1_0003,1,/content/drive/MyDrive/CognitivaAI/oas1_data/O...
1,OAS1_0003,1,/content/drive/MyDrive/CognitivaAI/oas1_data/O...
2,OAS1_0003,1,/content/drive/MyDrive/CognitivaAI/oas1_data/O...



Ejemplo holdout_df:


,patient_id,y_true,png_path
0,OAS1_0021,1,/content/drive/MyDrive/CognitivaAI/oas1_data/O...
1,OAS1_0021,1,/content/drive/MyDrive/CognitivaAI/oas1_data/O...
2,OAS1_0021,1,/content/drive/MyDrive/CognitivaAI/oas1_data/O...



Ejemplo test_df:


,patient_id,y_true,png_path
0,OAS1_0002,0,/content/drive/MyDrive/CognitivaAI/oas1_data/O...
1,OAS1_0002,0,/content/drive/MyDrive/CognitivaAI/oas1_data/O...
2,OAS1_0002,0,/content/drive/MyDrive/CognitivaAI/oas1_data/O...


In [6]:
# === Celda 3: Dataset + Modelo (EffNet-B3) + Entrenamiento estable con early stopping ===
import os, math, json, time, random, shutil
from pathlib import Path
from dataclasses import dataclass
from typing import Tuple, Dict, Any

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm.auto import tqdm

try:
    import timm
except ImportError:
    !pip -q install timm
    import timm

from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, precision_score, recall_score

# --- reutilizamos cfg (ya existe) y rutas (OUT_DIR, GRAPHS, etc.) ---
SEEDS = list(cfg.seeds) if hasattr(cfg, "seeds") else [42]
IMG_SIZE = int(cfg.img_size)
BATCH    = int(cfg.batch_size)
NUMW     = int(cfg.num_workers)

OUT_DIR = Path(OUT_DIR)  # ya definido arriba en tu Celda 1
OUT_DIR.mkdir(parents=True, exist_ok=True)
CKPT_BEST = OUT_DIR / "best_effb3_stable.pth"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# ----------------------------
# Dataset / transforms
# ----------------------------
MEAN = [0.485, 0.456, 0.406]
STD  = [0.229, 0.224, 0.225]

train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    # Augmentaciones suaves y deterministas entre seeds:
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomApply([transforms.RandomRotation(degrees=10)], p=0.3),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])

eval_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])

class SliceDataset(Dataset):
    def __init__(self, df: pd.DataFrame, tfm):
        assert {'patient_id', 'y_true', 'png_path'}.issubset(df.columns), \
            "El DataFrame debe tener columnas: patient_id, y_true, png_path"
        self.df  = df.reset_index(drop=True)
        self.tfm = tfm

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row   = self.df.iloc[idx]
        path  = row['png_path']
        label = int(row['y_true'])
        img = Image.open(path).convert("RGB")
        img = self.tfm(img)
        return img, label, row['patient_id'], path

# ----------------------------
# Modelo EffNet-B3 (timm)
# ----------------------------
def create_model(num_classes=1):
    model = timm.create_model("tf_efficientnet_b3_ns", pretrained=True, in_chans=3, num_classes=num_classes)
    # num_classes=1 → logits para BCEWithLogitsLoss
    return model

# ----------------------------
# Utilidades
# ----------------------------
@dataclass
class TrainCfg:
    epochs: int = 10
    lr: float = 1e-4
    wd: float = 1e-5
    amp: bool = True
    patience: int = 3   # early stop en holdout AUC
    label_smoothing: float = 0.0

train_cfg = TrainCfg(epochs=8, lr=1e-4, wd=1e-5, amp=True, patience=3, label_smoothing=0.0)
print(train_cfg)

bce_logits = nn.BCEWithLogitsLoss(reduction="mean")

def loss_fn(logits, targets, ls=0.0):
    # opcional: label smoothing binario
    if ls > 0:
        targets = targets.float() * (1 - ls) + 0.5 * ls
    return bce_logits(logits.view(-1), targets.float())

@torch.no_grad()
def evaluate_slices(model, loader) -> Dict[str, float]:
    model.eval()
    all_y = []
    all_p = []
    for imgs, ys, _, _ in loader:
        imgs = imgs.to(device, non_blocking=True)
        ys   = ys.to(device, non_blocking=True)
        with torch.amp.autocast('cuda', enabled=(device.type=='cuda' and train_cfg.amp)):
            logits = model(imgs).view(-1)
            probs  = torch.sigmoid(logits)
        all_y.append(ys.detach().float().cpu().numpy())
        all_p.append(probs.detach().float().cpu().numpy())
    y_true  = np.concatenate(all_y)
    y_score = np.concatenate(all_p)
    try:
        auc   = roc_auc_score(y_true, y_score)
        prauc = average_precision_score(y_true, y_score)
    except ValueError:
        auc, prauc = np.nan, np.nan
    return {"AUC": float(auc), "PR-AUC": float(prauc)}

def train_one_seed(seed: int,
                   train_df: pd.DataFrame,
                   holdout_df: pd.DataFrame,
                   train_cfg: TrainCfg,
                   save_path: Path) -> Dict[str, Any]:
    # reproducibilidad
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if device.type == "cuda":
        torch.cuda.manual_seed_all(seed)

    # datasets + loaders
    ds_train   = SliceDataset(train_df,   train_tfms)
    ds_holdout = SliceDataset(holdout_df, eval_tfms)

    dl_train   = DataLoader(ds_train,   batch_size=BATCH, shuffle=True,
                            num_workers=NUMW, pin_memory=True, drop_last=True, persistent_workers=False)
    dl_holdout = DataLoader(ds_holdout, batch_size=BATCH, shuffle=False,
                            num_workers=NUMW, pin_memory=True, drop_last=False, persistent_workers=False)

    model = create_model().to(device)
    opt   = torch.optim.AdamW(model.parameters(), lr=train_cfg.lr, weight_decay=train_cfg.wd)
    scaler = torch.amp.GradScaler('cuda', enabled=(device.type=="cuda" and train_cfg.amp))
    best_auc = -1.0
    best_sd  = None
    epochs_no_improve = 0

    pbar = tqdm(range(train_cfg.epochs), desc=f"Seed {seed} | training", leave=False)
    for epoch in pbar:
        model.train()
        running = 0.0
        for imgs, ys, _, _ in tqdm(dl_train, desc=f"Epoch {epoch+1}/{train_cfg.epochs}", leave=False):
            imgs = imgs.to(device, non_blocking=True)
            ys   = ys.to(device, non_blocking=True)

            opt.zero_grad(set_to_none=True)
            with torch.amp.autocast('cuda', enabled=(device.type=='cuda' and train_cfg.amp)):
                logits = model(imgs).view(-1)
                loss   = loss_fn(logits, ys, ls=train_cfg.label_smoothing)

            scaler.scale(loss).hfulf = None  # (no-op para dejar claro que usamos scaler)
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()
            running += loss.item()

        # eval holdout (slices)
        hold_metrics = evaluate_slices(model, dl_holdout)
        pbar.set_postfix(loss=f"{running/max(1,len(dl_train)):.4f}",
                         AUC=f"{hold_metrics['AUC']:.3f}",
                         PRAUC=f"{hold_metrics['PR-AUC']:.3f}")

        # early stopping por AUC holdout
        if hold_metrics["AUC"] > best_auc:
            best_auc = hold_metrics["AUC"]
            best_sd  = {k: v.detach().cpu() if isinstance(v, torch.Tensor) else v
                        for k, v in model.state_dict().items()}
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1
            if epochs_no_improve >= train_cfg.patience:
                print(f"⏹️ Early stopping en epoch {epoch+1} | best AUC holdout={best_auc:.3f}")
                break

    # guardar mejor checkpoint
    if best_sd is not None:
        torch.save({"seed": seed, "state_dict": best_sd, "best_holdout_auc": best_auc}, save_path)
        print(f"💾 Guardado mejor checkpoint (seed {seed}) en: {save_path} | AUC holdout={best_auc:.3f}")
    else:
        print("⚠️ No se guardó checkpoint (no hubo mejora).")

    return {"seed": seed, "best_holdout_auc": best_auc, "ckpt_path": str(save_path)}

# ----------------------------
# Entrenamiento (todas las seeds)
# ----------------------------
ckpt_paths = []
history = []
for s in SEEDS:
    ckpt_s = OUT_DIR / f"effb3_stable_seed{s}.pth"
    info = train_one_seed(s, train_df, holdout_df, train_cfg, ckpt_s)
    ckpt_paths.append(str(ckpt_s))
    history.append(info)

# seleccionar el mejor por AUC holdout (si hay varias seeds)
best = max(history, key=lambda d: d.get("best_holdout_auc", -1))
best_src = Path(best["ckpt_path"])
if best_src.exists():
    shutil.copy2(best_src, CKPT_BEST)
print("✅ Checkpoints:", ckpt_paths)
print("🏆 Mejor (por holdout AUC):", best)
print("➡️ Copiado como BEST:", CKPT_BEST)

# Guardar pequeño JSON con metadatos de entrenamiento
meta = {
    "cfg": {
        "img_size": IMG_SIZE,
        "batch_size": BATCH,
        "num_workers": NUMW,
        "seeds": SEEDS,
        "epochs": train_cfg.epochs,
        "lr": train_cfg.lr,
        "weight_decay": train_cfg.wd,
        "amp": train_cfg.amp,
        "patience": train_cfg.patience
    },
    "checkpoints": history,
    "best_ckpt": str(CKPT_BEST)
}
with open(OUT_DIR / "train_history_stable.json", "w", encoding="utf-8") as f:
    json.dump(meta, f, indent=2)
print("📝 train_history_stable.json guardado.")


Device: cuda
TrainCfg(epochs=8, lr=0.0001, wd=1e-05, amp=True, patience=3, label_smoothing=0.0)


/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/49.3M [00:00<?, ?B/s]

Seed 42 | training:   0%|          | 0/8 [00:00<?, ?it/s]

Epoch 1/8:   0%|          | 0/11 [00:00<?, ?it/s]

...

Epoch 6/8:   0%|          | 0/11 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7cbec8bcc9a0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
    if w.is_alive():
       ^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
           Exception ignored in: ^<function _MultiProcessingDataLoaderIter.__del__ at 0x7cbec8bcc9a0>
^^Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1664, in __del__
^    self._shutdown_workers()^
^  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1647, in _shutdown_workers
^    ^if w.is_alive():^
^ ^ ^ ^ ^ ^ ^ ^^^^^^^^^

Epoch 7/8:   0%|          | 0/11 [00:00<?, ?it/s]

Epoch 8/8:   0%|          | 0/11 [00:00<?, ?it/s]

💾 Guardado mejor checkpoint (seed 42) en: /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/effb3_stable_seed42.pth | AUC holdout=0.776
✅ Checkpoints: ['/content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/effb3_stable_seed42.pth']
🏆 Mejor (por holdout AUC): {'seed': 42, 'best_holdout_auc': 0.77575, 'ckpt_path': '/content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/effb3_stable_seed42.pth'}
➡️ Copiado como BEST: /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/best_effb3_stable.pth
📝 train_history_stable.json guardado.


In [7]:
# Pipeline 9 — Celda 4: inferencia + calibración + agregación paciente + métricas + guardado
import json, time, numpy as np, pandas as pd, matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix
from pathlib import Path
from tqdm.auto import tqdm
import torch, torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from PIL import Image
import timm

# Reutilizamos: OUT_DIR, GRAPHS_DIR, train_df, holdout_df, test_df, MEAN, STD, IMG_SIZE, NUMW, BATCH, device
BEST_CKPT = OUT_DIR / "best_effb3_stable.pth"
assert BEST_CKPT.exists(), f"No encuentro el checkpoint: {BEST_CKPT}"

# Dataset y eval transforms
eval_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(MEAN, STD),
])

class SliceDatasetEval(torch.utils.data.Dataset):
    def __init__(self, df: pd.DataFrame):
        self.df = df.reset_index(drop=True)
    def __len__(self): return len(self.df)
    def __getitem__(self, i):
        row = self.df.iloc[i]
        img = Image.open(row["png_path"]).convert("RGB")
        img = eval_tfms(img)
        return img, int(row["y_true"]), row["patient_id"]

dl_hold = DataLoader(SliceDatasetEval(holdout_df), batch_size=BATCH, shuffle=False,
                     num_workers=NUMW, pin_memory=True, drop_last=False, persistent_workers=False)
dl_test = DataLoader(SliceDatasetEval(test_df),    batch_size=BATCH, shuffle=False,
                     num_workers=NUMW, pin_memory=True, drop_last=False, persistent_workers=False)

# Modelo
def create_model(num_classes=1):
    m = timm.create_model("tf_efficientnet_b3_ns", pretrained=False, num_classes=num_classes, in_chans=3)
    return m.to(device)

ckpt = torch.load(BEST_CKPT, map_location=device)
model = create_model(num_classes=1)
sd = ckpt["state_dict"] if "state_dict" in ckpt else ckpt
model.load_state_dict(sd)
model.eval()

@torch.inference_mode()
def predict_logit_pid(dataloader):
    logits_all, labels_all, pids_all = [], [], []
    t0 = time.time()
    for xb, yb, pids in tqdm(dataloader, leave=False):
        xb = xb.to(device, non_blocking=True)
        with torch.amp.autocast('cuda', enabled=(device.type=='cuda')):
            logits = model(xb).squeeze(1)
        logits_all.append(logits.float().cpu().numpy())
        labels_all.append(yb.numpy().astype(int))
        pids_all.extend(list(pids))
    dt = time.time() - t0
    logits = np.concatenate(logits_all) if logits_all else np.array([])
    labels = np.concatenate(labels_all) if labels_all else np.array([])
    thp = len(labels) / max(dt, 1e-6)
    return logits, labels, np.array(pids_all), thp

val_logits, val_y, val_pids, thp_val = predict_logit_pid(dl_hold)
test_logits, test_y, test_pids, thp_test = predict_logit_pid(dl_test)
print(f"[VAL] throughput ≈ {thp_val:.1f} img/s | n_slices={len(val_y)}")
print(f"[TEST] throughput ≈ {thp_test:.1f} img/s | n_slices={len(test_y)}")

# ----- Temperature scaling (fit en HOLDOUT) -----
class TempScale(nn.Module):
    def __init__(self, T_init=1.0):
        super().__init__()
        self.T = nn.Parameter(torch.tensor(float(T_init)))
    def forward(self, z): return z / self.T.clamp(min=1e-3)

def fit_temperature(logits_np, labels_np, T_init=1.0, steps=200):
    x = torch.tensor(logits_np, dtype=torch.float32, device=device)
    y = torch.tensor(labels_np, dtype=torch.float32, device=device)
    modelT = TempScale(T_init).to(device)
    opt = torch.optim.LBFGS(modelT.parameters(), lr=0.01, max_iter=steps)
    bce = nn.BCEWithLogitsLoss()
    def closure():
        opt.zero_grad(set_to_none=True)
        loss = bce(modelT(x), y)
        loss.backward()
        return loss
    opt.step(closure)
    return float(modelT.T.detach().cpu().item())

T = fit_temperature(val_logits, val_y, T_init=1.0, steps=200)
print(f"✅ Temperature fitted: T = {T:.4f}")

sigmoid = lambda z: 1 / (1 + np.exp(-z))
val_prob  = sigmoid(val_logits  / T)
test_prob = sigmoid(test_logits / T)

# ----- Agregación nivel paciente -----
def aggregate_patient(pids, y_true, prob, method="mean"):
    df = pd.DataFrame({"patient_id": pids, "y_true": y_true, "y_score": prob})
    g = df.groupby("patient_id")
    if method=="mean": s = g["y_score"].mean()
    elif method=="max": s = g["y_score"].max()
    elif method=="median": s = g["y_score"].median()
    else: raise ValueError("pooling no soportado")
    y = g["y_true"].mean().round().astype(int)
    out = pd.DataFrame({"patient_id": s.index, "y_true": y.values, "y_score": s.values})
    return df, out

POOLING = "mean"
val_slices_df,  val_patient_df  = aggregate_patient(val_pids,  val_y,  val_prob,  method=POOLING)
test_slices_df, test_patient_df = aggregate_patient(test_pids, test_y, test_prob, method=POOLING)

# ----- Selección de umbral: mejor F1 con recall>=0.95 en VAL -----
def pick_threshold(df, force_recall=0.95):
    y = df.y_true.values
    s = df.y_score.values
    thrs = np.linspace(0.05, 0.95, 181)
    best_thr, best_f1 = None, -1
    for thr in thrs:
        pred = (s>=thr).astype(int)
        tp = ((pred==1)&(y==1)).sum()
        fp = ((pred==1)&(y==0)).sum()
        fn = ((pred==0)&(y==1)).sum()
        prec = tp / (tp+fp+1e-9)
        rec  = tp / (tp+fn+1e-9)
        if rec < force_recall:  # exigimos sensibilidad alta
            continue
        f1 = 2*prec*rec/(prec+rec+1e-9)
        if f1 > best_f1:
            best_f1, best_thr = f1, thr
    if best_thr is None:
        best_thr = 0.3651449978351593  # fallback conocido
    return float(best_thr)

thr = pick_threshold(val_patient_df, force_recall=0.95)
print(f"🔎 Umbral seleccionado (VAL, recall≥0.95): thr={thr:.4f}")

def patient_metrics(df, thr):
    y = df.y_true.values; s = df.y_score.values
    auc = roc_auc_score(y, s); pr = average_precision_score(y, s)
    pred = (s>=thr).astype(int)
    tn, fp, fn, tp = confusion_matrix(y, pred).ravel()
    acc = (tp+tn)/len(y)
    P = tp/(tp+fp+1e-9); R = tp/(tp+fn+1e-9)
    return dict(AUC=float(auc), PR_AUC=float(pr), Acc=float(acc), P=float(P), R=float(R),
                thr=float(thr), n=int(len(y)), TP=int(tp), FP=int(fp), TN=int(tn), FN=int(fn))

VAL_MET  = patient_metrics(val_patient_df, thr)
TEST_MET = patient_metrics(test_patient_df, thr)
print("VAL :", VAL_MET)
print("TEST:", TEST_MET)

# ----- Guardar CSVs y JSON de evaluación -----
val_slices_csv   = OUT_DIR / "val_slices_preds.csv"
test_slices_csv  = OUT_DIR / "test_slices_preds.csv"
val_patient_csv  = OUT_DIR / "val_patient_preds.csv"
test_patient_csv = OUT_DIR / "test_patient_preds.csv"
eval_json        = OUT_DIR / "effb3_stable_patient_eval.json"

val_slices_df.to_csv(val_slices_csv, index=False)
test_slices_df.to_csv(test_slices_csv, index=False)
val_patient_df.to_csv(val_patient_csv, index=False)
test_patient_df.to_csv(test_patient_csv, index=False)

payload = {
    "pooling_used": POOLING,
    "temperature": float(T),
    "threshold": float(thr),
    "val_metrics": VAL_MET,
    "test_metrics": TEST_MET,
}
with open(eval_json, "w", encoding="utf-8") as f:
    json.dump(payload, f, indent=2)

print("📄 Guardados:")
print(" -", val_slices_csv)
print(" -", test_slices_csv)
print(" -", val_patient_csv)
print(" -", test_patient_csv)
print(" -", eval_json)

# ----- Gráficas rápidas -----
def bar_metric(d, title, fname):
    plt.figure(figsize=(5,3))
    keys = ["AUC","PR_AUC","Acc","P","R"]
    vals = [d[k] for k in keys]
    plt.bar(keys, vals)
    plt.ylim(0,1.05)
    plt.title(title)
    plt.grid(axis="y", alpha=0.3)
    plt.tight_layout()
    path = GRAPHS_DIR / fname
    plt.savefig(path, dpi=150); plt.close()
    return path

b1 = bar_metric(VAL_MET,  "VAL metrics (patient)",  "effb3_stable_val_bars.png")
b2 = bar_metric(TEST_MET, "TEST metrics (patient)", "effb3_stable_test_bars.png")

def pr_point(df, title, fname, thr):
    y, s = df.y_true.values, df.y_score.values
    pred = (s>=thr).astype(int)
    tp = ((pred==1)&(y==1)).sum(); fp = ((pred==1)&(y==0)).sum(); fn = ((pred==0)&(y==1)).sum()
    prec = tp/(tp+fp+1e-9); rec = tp/(tp+fn+1e-9)
    plt.figure(figsize=(4,4))
    plt.scatter([rec],[prec], s=120)
    plt.xlim(0,1); plt.ylim(0,1)
    plt.xlabel("Recall"); plt.ylabel("Precision")
    plt.title(title + f"\nthr={thr:.3f}")
    plt.grid(alpha=0.3)
    plt.tight_layout()
    path = GRAPHS_DIR / fname
    plt.savefig(path, dpi=150); plt.close()
    return path

p_val  = pr_point(val_patient_df,  "PR point (VAL)",  "effb3_stable_pr_val.png",  thr)
p_test = pr_point(test_patient_df, "PR point (TEST)", "effb3_stable_pr_test.png", thr)

def plot_confusion(d, title, fname):
    tp, fp, tn, fn = d["TP"], d["FP"], d["TN"], d["FN"]
    mat = np.array([[tn, fp],[fn, tp]])
    plt.figure(figsize=(4,4))
    plt.imshow(mat, cmap="Blues")
    for (i,j),v in np.ndenumerate(mat):
        plt.text(j, i, str(v), ha="center", va="center", fontsize=14)
    plt.xticks([0,1],["Pred 0","Pred 1"])
    plt.yticks([0,1],["True 0","True 1"])
    plt.title(title)
    plt.tight_layout()
    path = GRAPHS_DIR / fname
    plt.savefig(path, dpi=150); plt.close()
    return path

c_val  = plot_confusion(VAL_MET,  "Confusion (VAL)",  "effb3_stable_conf_val.png")
c_test = plot_confusion(TEST_MET, "Confusion (TEST)", "effb3_stable_conf_test.png")

print("🖼️ Gráficas en:", GRAPHS_DIR)
for p in (b1,b2,p_val,p_test,c_val,c_test):
    print(" -", p)


/usr/local/lib/python3.12/dist-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b3_ns to current tf_efficientnet_b3.ns_jft_in1k.
  model = create_fn(


  0%|          | 0/4 [00:00<?, ?it/s]

  0%|          | 0/15 [00:00<?, ?it/s]

[VAL] throughput ≈ 67.5 img/s | n_slices=200
[TEST] throughput ≈ 4.4 img/s | n_slices=940


/usr/local/lib/python3.12/dist-packages/torch/optim/lbfgs.py:457: UserWarning: Converting a tensor with requires_grad=True to a scalar may lead to unexpected behavior.
Consider using tensor.detach() first. (Triggered internally at /pytorch/torch/csrc/autograd/generated/python_variable_methods.cpp:835.)
  loss = float(closure())


✅ Temperature fitted: T = 2.0479
🔎 Umbral seleccionado (VAL, recall≥0.95): thr=0.3400
VAL : {'AUC': 1.0, 'PR_AUC': 1.0, 'Acc': 1.0, 'P': 0.9999999998, 'R': 0.9999999998, 'thr': 0.33999999999999997, 'n': 10, 'TP': 5, 'FP': 0, 'TN': 5, 'FN': 0}
TEST: {'AUC': 0.662962962962963, 'PR_AUC': 0.6797888758057811, 'Acc': 0.574468085106383, 'P': 0.4999999999807692, 'R': 0.6499999999675, 'thr': 0.33999999999999997, 'n': 47, 'TP': 13, 'FP': 13, 'TN': 14, 'FN': 7}
📄 Guardados:
 - /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/val_slices_preds.csv
 - /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/test_slices_preds.csv
 - /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/val_patient_preds.csv
 - /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/test_patient_preds.csv
 - /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/effb3_stable_patient_eval.json
🖼️ Gráficas en: /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/graphs_from_metrics
 - /content/drive/My

In [8]:
# Pipeline 9 — Celda 5: resumen + bloques Markdown con timestamp Europe/Madrid
from datetime import datetime
import pytz, json

tz = pytz.timezone("Europe/Madrid")
now = datetime.now(tz).strftime("%d/%m/%Y – %H:%M")

with open(OUT_DIR / "effb3_stable_patient_eval.json", "r", encoding="utf-8") as f:
    ej = json.load(f)
val, tes = ej.get("val_metrics", {}), ej.get("test_metrics", {})
pooling, T, thr = ej["pooling_used"], ej["temperature"], ej["threshold"]

def fmt(x, nd=3):
    try: return f"{float(x):.{nd}f}"
    except: return str(x)

val_line = f"VAL → AUC={fmt(val.get('AUC'))} | PR-AUC={fmt(val.get('PR_AUC'))} | Acc={fmt(val.get('Acc'))} | P={fmt(val.get('P'))} | R={fmt(val.get('R'))} | thr={fmt(val.get('thr'),4)} | n={val.get('n')}"
tes_line = f"TEST → AUC={fmt(tes.get('AUC'))} | PR-AUC={fmt(tes.get('PR_AUC'))} | Acc={fmt(tes.get('Acc'))} | P={fmt(tes.get('P'))} | R={fmt(tes.get('R'))} | thr={fmt(tes.get('thr'),4)} | n={tes.get('n')}"

readme_block = f"""### 9️⃣ COGNITIVA-AI-FINETUNING-STABLE (EfficientNet‑B3, Colab)
- **Notebook:** `cognitiva_ai_finetuning_stable.ipynb`
- **Pooling paciente:** {pooling}
- **Calibración:** temperature scaling (T={fmt(T)})
- **Umbral clínico:** {fmt(thr,4)} (selección en VAL con recall≥0.95)

**Resultados (nivel paciente):**
- {val_line}
- {tes_line}

**Gráficas:** ver `ft_effb3_stable_colab/graphs_from_metrics/`
- `effb3_stable_val_bars.png` / `effb3_stable_test_bars.png`
- `effb3_stable_pr_val.png` / `effb3_stable_pr_test.png`
- `effb3_stable_conf_val.png` / `effb3_stable_conf_test.png`

> _Última actualización README: {now} (Europe/Madrid)_
"""

bitacora_block = f"""### 📅 {now} – Pipeline 9 (EffB3 estable)
- **Acción:** retraining reproducible en Colab (EffNet‑B3), caché SSD, calibración (T={fmt(T)}), pooling `{pooling}`, umbral {fmt(thr,4)} con recall≥0.95 en VAL.
- **Resultados:**
  - {val_line}
  - {tes_line}
- **Artefactos:** `best_effb3_stable.pth`, `effb3_stable_patient_eval.json`, CSVs por slice y paciente, y gráficas en `graphs_from_metrics/`.
- **Conclusión:** setup estable listo para comparación con Pipelines 6–7 y para el salto a **multimodal**.
"""

informe_block = f"""## 9. Fine‑tuning Estable EfficientNet‑B3 (Colab)
**Configuración**
- Arquitectura: EfficientNet‑B3 (timm).
- Entrenamiento: AdamW (lr fijo), AMP (`torch.amp`), early‑stopping por AUC en holdout.
- Agregación: `{pooling}` a nivel paciente.
- Calibración: temperature scaling (T={fmt(T)}).
- Umbral: {fmt(thr,4)} (optimizado con recall≥0.95 en VAL).

**Resultados cuantitativos**
- {val_line}
- {tes_line}

**Artefactos**
- Checkpoint: `best_effb3_stable.pth`
- JSON: `effb3_stable_patient_eval.json`
- CSVs: `val/test_slices_preds.csv`, `val/test_patient_preds.csv`
- Gráficas: barras, punto PR y confusión (`graphs_from_metrics/`)

> _Última actualización InformeTécnico: {now} (Europe/Madrid)_
"""

print("\n" + "="*70 + "\nREADME.md — bloque para pegar\n" + "="*70 + "\n")
print(readme_block)
print("\n" + "="*70 + "\nCuadernoBitacora.md — bloque para pegar\n" + "="*70 + "\n")
print(bitacora_block)
print("\n" + "="*70 + "\nInformeTecnico.md — bloque para pegar\n" + "="*70 + "\n")
print(informe_block)



README.md — bloque para pegar

### 9️⃣ COGNITIVA-AI-FINETUNING-STABLE (EfficientNet‑B3, Colab)
- **Notebook:** `cognitiva_ai_finetuning_stable.ipynb`  
- **Pooling paciente:** mean  
- **Calibración:** temperature scaling (T=2.048)  
- **Umbral clínico:** 0.3400 (selección en VAL con recall≥0.95)

**Resultados (nivel paciente):**  
- VAL → AUC=1.000 | PR-AUC=1.000 | Acc=1.000 | P=1.000 | R=1.000 | thr=0.3400 | n=10  
- TEST → AUC=0.663 | PR-AUC=0.680 | Acc=0.574 | P=0.500 | R=0.650 | thr=0.3400 | n=47

**Gráficas:** ver `ft_effb3_stable_colab/graphs_from_metrics/`  
- `effb3_stable_val_bars.png` / `effb3_stable_test_bars.png`  
- `effb3_stable_pr_val.png` / `effb3_stable_pr_test.png`  
- `effb3_stable_conf_val.png` / `effb3_stable_conf_test.png`

> _Última actualización README: 25/08/2025 – 02:57 (Europe/Madrid)_


CuadernoBitacora.md — bloque para pegar

### 📅 25/08/2025 – 02:57 – Pipeline 9 (EffB3 estable)
- **Acción:** retraining reproducible en Colab (EffNet‑B3), caché SSD, calibr

In [9]:
# Celda 6 — Comparativa global P7 vs P9 (AUC y PR-AUC)
import json, os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt

# Rutas esperadas (ajústalas si las cambiaste)
P7_JSON = BASE / "ft_effb3_colab"         / "ft_effb3_patient_eval.json"         # pipeline 7 (tu FT previo)
P9_JSON = OUT_DIR                          / "effb3_stable_patient_eval.json"     # pipeline 9 (estable)

rows = []
for name, path in [("P7_FT_B3", P7_JSON), ("P9_FT_B3_STABLE", P9_JSON)]:
    if Path(path).exists():
        with open(path, "r", encoding="utf-8") as f:
            j = json.load(f)
        v, t = j.get("val_metrics", {}), j.get("test_metrics", {})
        rows.append({
            "pipeline": name,
            "T": j.get("temperature", None),
            "thr": j.get("threshold", None),
            "VAL_AUC": v.get("AUC", None),
            "VAL_PR_AUC": v.get("PR-AUC", v.get("PR_AUC", None)),
            "TEST_AUC": t.get("AUC", None),
            "TEST_PR_AUC": t.get("PR-AUC", t.get("PR_AUC", None)),
        })
    else:
        print(f"⚠️ No encuentro {name}: {path}")

dfc = pd.DataFrame(rows)
display(dfc)

# Gráficas
def _bar(ax, labels, vals, title):
    ax.bar(labels, vals)
    ax.set_ylim(0, 1.05)
    ax.set_title(title)
    ax.grid(axis="y", alpha=0.3)

fig, axs = plt.subplots(1, 2, figsize=(10,4))
_bar(axs[0], dfc["pipeline"], dfc["TEST_AUC"],    "TEST AUC — P7 vs P9")
_bar(axs[1], dfc["pipeline"], dfc["TEST_PR_AUC"], "TEST PR-AUC — P7 vs P9")
plt.tight_layout()
cmp1 = GRAPHS_DIR / "comparison_p7_p9_test.png"
plt.savefig(cmp1, dpi=150)
plt.close()

fig, axs = plt.subplots(1, 2, figsize=(10,4))
_bar(axs[0], dfc["pipeline"], dfc["VAL_AUC"],    "VAL AUC — P7 vs P9")
_bar(axs[1], dfc["pipeline"], dfc["VAL_PR_AUC"], "VAL PR-AUC — P7 vs P9")
plt.tight_layout()
cmp2 = GRAPHS_DIR / "comparison_p7_p9_val.png"
plt.savefig(cmp2, dpi=150)
plt.close()

print("🖼️ Comparativas guardadas en:")
print(" -", cmp1)
print(" -", cmp2)


,pipeline,T,thr,VAL_AUC,VAL_PR_AUC,TEST_AUC,TEST_PR_AUC
0,P7_FT_B3,2.302744,0.05,0.52381,0.433333,0.585185,0.582197
1,P9_FT_B3_STABLE,2.047882,0.34,1.00000,1.000000,0.662963,0.679789


🖼️ Comparativas guardadas en:
 - /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/graphs_from_metrics/comparison_p7_p9_test.png
 - /content/drive/MyDrive/CognitivaAI/ft_effb3_stable_colab/graphs_from_metrics/comparison_p7_p9_val.png


In [10]:
from datetime import datetime
import pytz, json

tz = pytz.timezone("Europe/Madrid")
now = datetime.now(tz).strftime("%d/%m/%Y – %H:%M")

def get_eval(path):
    if not Path(path).exists():
        return None
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)

ej9 = get_eval(OUT_DIR / "effb3_stable_patient_eval.json")
ej7 = get_eval(BASE    / "ft_effb3_colab" / "ft_effb3_patient_eval.json")

def fmt(x, nd=3):
    try: return f"{float(x):.{nd}f}"
    except: return str(x)

def line(name, d):
    return f"{name} → AUC={fmt(d.get('AUC'))} | PR-AUC={fmt(d.get('PR_AUC') or d.get('PR-AUC'))} | Acc={fmt(d.get('Acc'))} | P={fmt(d.get('P'))} | R={fmt(d.get('R'))} | thr={fmt(d.get('thr'),4)} | n={d.get('n')}"

pooling = ej9.get("pooling_used", "mean")
T9      = ej9.get("temperature", None)
thr9    = ej9.get("threshold", None)
VAL9    = ej9.get("val_metrics", {})
TEST9   = ej9.get("test_metrics", {})

readme_block = f"""### 9️⃣ COGNITIVA-AI-FINETUNING-STABLE (EfficientNet‑B3, Colab)
- **Notebook:** `cognitiva_ai_finetuning_stable.ipynb`
- **Pooling paciente:** {pooling}
- **Calibración:** temperature scaling (T={fmt(T9)})
- **Umbral clínico:** {fmt(thr9,4)} (selección en VAL con recall≥0.95)

**Resultados (nivel paciente):**
- {line("VAL", VAL9)}
- {line("TEST", TEST9)}

{"**Comparativa rápida vs Pipeline 7 (FT previo):** TEST AUC: "
 + f"{fmt(ej7['test_metrics']['AUC'])} → {fmt(TEST9['AUC'])}, TEST PR‑AUC: "
 + f"{fmt(ej7['test_metrics'].get('PR_AUC') or ej7['test_metrics'].get('PR-AUC'))} → {fmt(TEST9.get('PR_AUC') or TEST9.get('PR-AUC'))}" if ej7 else ""}

**Gráficas:** `ft_effb3_stable_colab/graphs_from_metrics/`
- `effb3_stable_val_bars.png` / `effb3_stable_test_bars.png`
- `effb3_stable_pr_val.png` / `effb3_stable_pr_test.png`
- `effb3_stable_conf_val.png` / `effb3_stable_conf_test.png`
- `comparison_p7_p9_test.png` / `comparison_p7_p9_val.png`

> _Última actualización README: {now} (Europe/Madrid)_
"""

bitacora_block = f"""### 📅 {now} – Pipeline 9 (EffB3 estable)
- **Acción:** retraining reproducible en Colab (EffNet‑B3), caché SSD, AMP (`torch.amp`), early‑stopping por AUC en holdout, calibración (T={fmt(T9)}), pooling `{pooling}` y selección de umbral {fmt(thr9,4)} con recall≥0.95 en VAL.
- **Resultados:**
  - {line("VAL", VAL9)}
  - {line("TEST", TEST9)}
{"- **Comparativa con P7:** ver `comparison_p7_p9_*` (AUC/PR‑AUC)." if ej7 else ""}
- **Artefactos:** `best_effb3_stable.pth`, `effb3_stable_patient_eval.json`, CSVs por slice/paciente y gráficas en `graphs_from_metrics/`.
- **Conclusión:** setup estable listo para el salto a **multimodal** y validación externa.
"""

informe_block = f"""## 9. Fine‑tuning Estable EfficientNet‑B3 (Colab)
**Configuración**
- Arquitectura: EfficientNet‑B3 (timm).
- Entrenamiento: AdamW (lr=1e‑4), AMP (`torch.amp`), early‑stopping por AUC en holdout, {cfg.img_size}px, batch={cfg.batch_size}.
- Agregación: `{pooling}` a nivel paciente.
- Calibración: temperature scaling (T={fmt(T9)}).
- Umbral: {fmt(thr9,4)} (optimizado con recall≥0.95 en VAL).

**Resultados cuantitativos**
- {line("VAL", VAL9)}
- {line("TEST", TEST9)}

**Comparativa**
{"- Frente al Pipeline 7 (FT previo): mejora/variación en TEST AUC/PR‑AUC reflejada en las gráficas `comparison_p7_p9_*`." if ej7 else "- Únicamente P9 disponible para comparar."}

**Artefactos**
- Checkpoint: `best_effb3_stable.pth`
- JSON: `effb3_stable_patient_eval.json`
- CSVs: `val/test_slices_preds.csv`, `val/test_patient_preds.csv`
- Gráficas: barras, punto PR y confusión, y comparativas P7 vs P9 (`graphs_from_metrics/`)

> _Última actualización InformeTécnico: {now} (Europe/Madrid)_
"""

print("\n" + "="*70 + "\nREADME.md — bloque para pegar\n" + "="*70 + "\n")
print(readme_block)
print("\n" + "="*70 + "\nCuadernoBitacora.md — bloque para pegar\n" + "="*70 + "\n")
print(bitacora_block)
print("\n" + "="*70 + "\nInformeTecnico.md — bloque para pegar\n" + "="*70 + "\n")
print(informe_block)



README.md — bloque para pegar

### 9️⃣ COGNITIVA-AI-FINETUNING-STABLE (EfficientNet‑B3, Colab)
- **Notebook:** `cognitiva_ai_finetuning_stable.ipynb`  
- **Pooling paciente:** mean  
- **Calibración:** temperature scaling (T=2.048)  
- **Umbral clínico:** 0.3400 (selección en VAL con recall≥0.95)

**Resultados (nivel paciente):**  
- VAL → AUC=1.000 | PR-AUC=1.000 | Acc=1.000 | P=1.000 | R=1.000 | thr=0.3400 | n=10  
- TEST → AUC=0.663 | PR-AUC=0.680 | Acc=0.574 | P=0.500 | R=0.650 | thr=0.3400 | n=47

**Comparativa rápida vs Pipeline 7 (FT previo):** TEST AUC: 0.585 → 0.663, TEST PR‑AUC: 0.582 → 0.680

**Gráficas:** `ft_effb3_stable_colab/graphs_from_metrics/`  
- `effb3_stable_val_bars.png` / `effb3_stable_test_bars.png`  
- `effb3_stable_pr_val.png` / `effb3_stable_pr_test.png`  
- `effb3_stable_conf_val.png` / `effb3_stable_conf_test.png`  
- `comparison_p7_p9_test.png` / `comparison_p7_p9_val.png`

> _Última actualización README: 25/08/2025 – 03:04 (Europe/Madrid)_


CuadernoBita